In [0]:
dbutils.fs.ls('/mnt/databricks/sleeper/stg/drafts/_year=2025/_matchup_week=0')

In [0]:
par_tables = dbutils.widgets.get('Tables').split(',')
par_years = dbutils.widgets.get('Years').split(',')
par_matchup_weeks = dbutils.widgets.get('MatchupWeeks').split(',')
par_filters = [filter.strip() for filter in dbutils.widgets.get('Filters').split(',')]
par_mode = dbutils.widgets.get('_Mode')

In [0]:
def move_files(source_path, target_path, file_filters):
    source_path =  source_path + ('/' if source_path[-1] != '/' else '')
    target_path =  target_path + ('/' if target_path[-1] != '/' else '')
    try:
        files = dbutils.fs.ls(source_path)
        for file in files:
            if all(filter in file.name for filter in file_filters) or len(file_filters) == 0:
                target_file_name = target_path + ('/' if target_path[-1] != '/' else '') + file.name
                dbutils.fs.mv(file.path, target_file_name)
                print(f'Moved {file.path.replace("dbfs:", "")} to {target_file_name}')
    except Exception as e:
        print(f'Error moving files: {str(e).split(":")[2]}')

In [0]:
def archive_files(table, year, matchup_week):
    source_path = f'/mnt/databricks/sleeper/stg/{table}/_year={year}/_matchup_week={matchup_week}'
    target_path = source_path.replace('stg', 'archive')

    move_files(source_path, target_path, par_filters)

def restore_files(table, year, matchup_week):
    source_path = f'/mnt/databricks/sleeper/archive/{table}/_year={year}/_matchup_week={matchup_week}'
    target_path = source_path.replace('archive', 'stg')

    move_files(source_path, target_path, par_filters)

for year in par_years:
    for table in par_tables:
        for matchup_week in par_matchup_weeks:

            if par_mode == 'restore':
                restore_files(table, year, matchup_week)
            elif par_mode == 'archive':
                archive_files(table, year, matchup_week)
            else:
                raise ValueError('Invalid mode: "{par_mode}" provided. Valid modes are "archive" and "restore"')


In [0]:
# for directory in dbutils.fs.ls('mnt/databricks/sleeper/archive/'):
#     files = dbutils.fs.ls(directory.path)

#     for file in files:
#         file_path = file.path.replace('dbfs:', '')
#         target_path = file_path.replace('2024', '2024/')
#         print(target_path)
#         dbutils.fs.mv(file_path, target_path)
#         print(f'Moved {file_path} to {target_path}')
        

In [0]:
# dirs = dbutils.fs.ls('/mnt/databricks/sleeper/stg/')
# for dir in dirs:
#     years = dbutils.fs.ls(dir.path)
#     years = [year for year in years if year.name == '2024/']
#     for year in years:
#         for matchup_week in dbutils.fs.ls(year.path):
#             for records in dbutils.fs.ls(matchup_week.path):
#                 target_path = records.path.replace('2024/', '_year=2024/').replace('1/', '_matchup_week=1/')

#                 print(f'Moving {records.path} to {target_path}')
#                 dbutils.fs.mv(
#                     records.path,
#                     target_path
#                 )
        
#         print(f'Removing dir: {year.path}')
#         dbutils.fs.rm(year.path, recurse=True)